# calc plane

In [1]:
import numpy as np
from leicacam import CAM

In [2]:
cam = CAM()

In [145]:
def get_position():
    info = cam.get_information()
    return np.array((float(info['xpos'])*1e6, float(info['ypos'])*1e6, float(info['zpos'])*1e6))

In [146]:
get_position()

array([ 56218.5064,  30859.2713,      0.    ])

In [28]:
p1 = get_position()

In [29]:
p2 = get_position()

In [30]:
p3 = get_position()

In [31]:
p4 = get_position()

In [32]:
p5 = get_position()

In [33]:
pL = get_position()

```
1            5
2            4
3            L
```

In [34]:
v15 = p5-p1
v24 = p4-p2
v3L = pL-p3
v15, v24, v3L

(array([  2.26605548e+04,   1.86166900e+02,  -1.78904000e+01]),
 array([  2.10973231e+04,   2.72543800e+02,  -4.19520000e+00]),
 array([  2.12689979e+04,   1.29457900e+02,   3.42760000e+00]))

In [35]:
v13 = p3-p1
v5L = pL-p5
v13, v5L

(array([  1613.9494,  13199.9786,    268.8234]),
 array([   222.3925,  13143.2696,    290.1414]))

In [37]:
imgsize = 1550 # microns
fielddist = imgsize * 0.90 # overlap
fielddist

1395.0

In [38]:
from IPython import display
resolution = 256
precision = (imgsize) // resolution # integer devision, in microns
display.Math(str(precision) + '\mu m') 

<IPython.core.display.Math object>

In [40]:
xmin = p1[0] # microns
ymin = p1[1]
xmax = pL[0]
ymax = pL[1]

In [41]:
dx = xmax-xmin
dy = ymax-ymin
dx, dy

(22882.9473, 13329.436499999996)

In [42]:
xsize = dx + imgsize/1000
ysize = dy + imgsize/1000
xsize, ysize

(22884.497299999999, 13330.986499999995)

In [44]:
xfields = round(xsize / fielddist)
yfields = round(ysize / fielddist)
xfields, yfields

(16.0, 10.0)

In [45]:
cam.socket.close()

# automated scan

In [46]:
from leicaautomator.automator import find_spots
from leicaautomator.utils import save_regions, flatten
from leicaexperiment import Experiment
from leicascanningtemplate import ScanningTemplate
from leicacam import CAM

In [280]:
def waitfor(cam, key, value=None):
    from time import sleep
    while True:
        msgs = cam.receive()
        for msg in msgs:
            if value and msg.get(key) == value:
                return msg
            elif not value and msg.get(key):
                return msg
        sleep(10e-3)

## rotate images

In [52]:
whole = 'experiment--2015_03_25_08_17_03'
exp_path = r"D:\MatrixScreenerImages\3.3.0_9757\ ".strip()
experiment = Experiment(exp_path + whole)
!dir $experiment.path

 Volume in drive D is DATADRIVE1
 Volume Serial Number is F805-2E08

 Directory of D:\MatrixScreenerImages\3.3.0_9757\experiment--2015_03_25_08_17_03

03/25/2015  09:17 AM    <DIR>          .
03/25/2015  09:17 AM    <DIR>          ..
03/25/2015  09:17 AM    <DIR>          AdditionalData
03/25/2015  09:17 AM    <DIR>          slide--S00
               0 File(s)              0 bytes
               4 Dir(s)  1,686,751,453,184 bytes free


In [53]:
compressed = experiment.compress(delete_tif=True)
assert len(compressed) == len(experiment.images)

In [54]:
from PIL import Image
for img in experiment.images:
    i = Image.open(img)
    i = i.rotate(270)
    i.save(img)

In [55]:
experiment.stitch()

['D:\\MatrixScreenerImages\\3.3.0_9757\\experiment--2015_03_25_08_17_03\\stitched--U00--V00--C00--Z00.png']

## find regions

In [56]:
regions = find_spots(experiment)

TypeError: filter_image() takes 1 positional argument but 3 were given

In [58]:
labels, image, regions = regions

In [63]:
save_regions(regions, 'regions.json')

In [61]:
for r in regions:
    print(r.well_x, r.well_y, r.real_x, r.real_y)

2 0 0.05837773346275717 0.030771066715570936
4 0 0.061477733462757166 0.030813532468995593
5 0 0.06308233228858887 0.0308286988095044
6 0 0.06479916203418573 0.03083173207760616
3 0 0.05993379999896069 0.03083476534570792
7 0 0.06640982739622096 0.030843865150013205
9 0 0.06970395655473367 0.030910597048251953
1 0 0.05678830097743427 0.030934863193066042
0 0 0.05501383913790394 0.030965195874083654
11 0 0.0729586532279235 0.031004628359406552
10 0 0.07126002309093721 0.031016761431813596
8 0 0.06806295851168084 0.030874197831030816
12 0 0.07458145166236577 0.03112292581537524
13 0 0.0761678508795869 0.03127155595236154
14 0 0.07751462191676889 0.031323121510091485
4 1 0.061468633658451885 0.03245453051204843
6 1 0.06468996438252234 0.03250609606977837
3 1 0.05988223444123075 0.03250912933788013
1 1 0.05667910332577087 0.032521262410287174
2 1 0.058289768687806096 0.03252429567838894
7 1 0.06640982739622096 0.03252429567838894
8 1 0.06805689197547732 0.0325273289464907
5 1 0.06299740078

In [60]:
p1

array([ 54977.8039,  30884.2957,   5783.3606])

In [148]:
# IN METERS
def get_position():
    info = cam.get_information()
    return np.array((float(info['xpos']), float(info['ypos']), float(info['zpos'])))

In [62]:
tmpl_path = r"C:\Users\TCS-User\AppData\Roaming\Leica Microsystems\LAS X\MatrixScreener\ScanningTemplates\ ".strip()
tmpl_name = tmpl_path + '{ScanningTemplate}leicaautomator'

In [64]:
max_well_x = max(r.well_x for r in regions)
max_well_y = max(r.well_y for r in regions)

In [65]:
max_well_x, max_well_y

(15, 8)

In [150]:
cam.connect()
z = get_position()[2]

In [103]:
i = 0
j = 0
well0 = next(r for r in regions if r.well_x == i and r.well_y == j)

In [139]:
i = 1
j = 0
well = next(r for r in regions if r.well_x == i and r.well_y == j)

In [125]:
tmpl = ScanningTemplate(tmpl_name + str(0) + '.xml')
tmpl.move_well(1, 1, well.real_x, well.real_y)
tmpl.well_attrib(1, 1)['FieldZCoordinate'] = str(z)

for f in tmpl.fields:
    f.FieldZCoordinate = z

In [126]:
tmpl.write()
cam.load_template(tmpl.filename)

Scanning well (1,0)


[]

In [130]:
x = get_position()[0]

In [128]:
y = get_position()[1]

In [174]:
z = get_position()[2]

In [136]:
(well.real_x - x)*1e6

569.79457743427338

In [137]:
(well.real_y - y)*1e6

633.10139306604106

In [140]:
(well.real_x - well0.real_x)*1e6

1774.461839530332

In [141]:
well.real_x*1e6, well.real_y*1e6, z*1e6

(56788.30097743427, 30934.86319306604, 0.0)

### try real offset other well too

In [153]:
i = 6
j = 6
well = next(r for r in regions if r.well_x == i and r.well_y == j)

In [155]:
tmpl = ScanningTemplate(tmpl_name + str(0) + '.xml')
tmpl.move_well(1, 1, well.real_x, well.real_y)
tmpl.well_attrib(1, 1)['FieldZCoordinate'] = str(z)

for f in tmpl.fields:
    f.FieldZCoordinate = z

In [156]:
tmpl.write()
cam.load_template(tmpl.filename)

[]

In [164]:
x = get_position()[0]

In [163]:
y = get_position()[1]

In [165]:
z = get_position()[2]

In [167]:
(well.real_x - x)*1e6

465.84590757124676

In [168]:
(well.real_y - y)*1e6

517.30852633413701

### try scan of first well

In [170]:
i = 0
j = 0
well = next(r for r in regions if r.well_x == i and r.well_y == j)

In [171]:
tmpl = ScanningTemplate(tmpl_name + str(i) + '.xml')
tmpl.move_well(1, 1, well.real_x - 600e-6, well.real_y - 600e-6)
tmpl.well_attrib(1, 1)['FieldZCoordinate'] = str(z)

for f in tmpl.fields:
    f.FieldZCoordinate = z

In [172]:
tmpl.write()
cam.load_template(tmpl.filename)

[]

In [177]:
print('Scanning well ({},{})'.format(i,j))
cam.send([('cmd', 'autofocusscan')])

Scanning well (0,0)


[OrderedDict([('cli', 'python-leicacam'), ('app', 'matrix'), ('cmd', 'autofocusscan')])]

In [178]:
cam.start_scan()

[]

In [179]:
waitfor(cam, 'inf', 'scanfinished')

### next

In [196]:
cam.get_information()

OrderedDict([('app', 'matrix'), ('sys', '0'), ('dev', 'stage'), ('info_for', 'python-leicacam'), ('unit', 'meter'), ('xpos', '0.0566300003'), ('ypos', '0.0303339981'), ('zpos', '0')])

In [197]:
cam.send(b'/cmd:getinfo /dev:zdrive')

[OrderedDict([('app', 'matrix'), ('sys', '0'), ('dev', 'zdrive'), ('info_for', 'python-leicacam'), ('unit', 'meter'), ('zpos', '0')]),
 OrderedDict([('cli', 'python-leicacam'), ('app', 'matrix'), ('cmd', 'getinfo'), ('dev', 'zdrive')])]

In [198]:
z = 8647e-6

In [219]:
i = 4
j = 0
well = next(r for r in regions if r.well_x == i and r.well_y == j)

In [220]:
tmpl = ScanningTemplate(tmpl_name + str(i%2) + '.xml')
tmpl.move_well(1, 1, well.real_x - 652e-6, well.real_y - 652e-6)
tmpl.well_attrib(1, 1)['FieldZCoordinate'] = str(z)

for f in tmpl.fields:
    f.FieldZCoordinate = z

In [221]:
tmpl.write()
cam.load_template(tmpl.filename)
waitfor(cam, 'cmd', 'load')

In [222]:
cam.send([('cmd', 'autofocusscan')])
waitfor(cam, 'inf', 'scanfinished')

In [223]:
print('Scanning well ({},{})'.format(i,j))
cam.start_scan()
waitfor(cam, 'inf', 'scanfinished')

Scanning well (4,0)


### time one well

In [225]:
%%time
i = 5
j = 0

well = next(r for r in regions if r.well_x == i and r.well_y == j)
tmpl = ScanningTemplate(tmpl_name + str(i%2) + '.xml')
tmpl.move_well(1, 1, well.real_x - 652e-6, well.real_y - 652e-6)
tmpl.write()

cam.load_template(tmpl.filename)
waitfor(cam, 'cmd', 'load')

cam.send([('cmd', 'autofocusscan')])
waitfor(cam, 'inf', 'scanfinished')

print('Scanning well ({},{})'.format(i,j))
cam.start_scan()
waitfor(cam, 'inf', 'scanfinished')

Scanning well (5,0)
Wall time: 3min 32s


In [226]:
%%time
i = 6
j = 0

well = next(r for r in regions if r.well_x == i and r.well_y == j)
tmpl = ScanningTemplate(tmpl_name + str(i%2) + '.xml')
tmpl.move_well(1, 1, well.real_x - 652e-6, well.real_y - 652e-6)
tmpl.write()

cam.load_template(tmpl.filename)
waitfor(cam, 'cmd', 'load')

cam.send([('cmd', 'autofocusscan')])
waitfor(cam, 'inf', 'scanfinished')

Wall time: 32.7 s


- 2048

In [227]:
%%time
print('Scanning well ({},{})'.format(i,j))
cam.start_scan()
waitfor(cam, 'inf', 'scanfinished')

Scanning well (6,0)
Wall time: 2min 56s


- 1024

In [228]:
%%time
print('Scanning well ({},{})'.format(i,j))
cam.start_scan()
waitfor(cam, 'inf', 'scanfinished')

Scanning well (6,0)
Wall time: 1min 58s


- 1024, only pmt

In [229]:
%%time
print('Scanning well ({},{})'.format(i,j))
cam.start_scan()
waitfor(cam, 'inf', 'scanfinished')

Scanning well (6,0)
Wall time: 1min 56s


- 1024, 5x5

In [230]:
%%time
print('Scanning well ({},{})'.format(i,j))
cam.start_scan()
waitfor(cam, 'inf', 'scanfinished')

Scanning well (6,0)
Wall time: 1min 23s


- 512, 6x6

In [232]:
%%time
print('Scanning well ({},{})'.format(i,j))
cam.start_scan()
waitfor(cam, 'inf', 'scanfinished')

Scanning well (6,0)
Wall time: 1min 26s


### all

1550 - 246 = 1304
1304 / 2 = 652

In [244]:
from datetime import datetime
import sys

- j == 0 twice

In [20]:
r = [1,2,3]

In [21]:
r[::-1]

[3, 2, 1]

In [283]:
n = 1
for i in range(max_well_x):
    for j in range(max_well_y):
        # count downwards on every second column, scanning in zick zack
        if i % 2 == 1:
            j = -(j+1) % max_well_y

        well = next((r for r in regions if r.well_x == i and r.well_y == j), None)
        if not well:
            continue

        print('Scanning well ({},{})'.format(i,j), flush=True)

        # create template, alternate between tmpl_name0/1.xml because of
        # a bug in LASAF (not loading the same name twice)
        tmpl = ScanningTemplate(tmpl_name + str(n%2) + '.xml')
        n += 1

        tmpl.move_well(1, 1, well.real_x - 652e-6, well.real_y - 652e-6)
        tmpl.write()

        cam.load_template(tmpl.filename)
        waitfor(cam, 'cmd', 'load')
        
        cam.send([('cmd', 'autofocusscan')])
        waitfor(cam, 'inf', 'scanfinished')
             
        cam.start_scan()
        print(datetime.now())
        print(waitfor(cam, 'relpath')['relpath'], flush=True)

        waitfor(cam, 'inf', 'scanfinished')

Scanning well (0,0)
2015-03-25 15:05:37.463110
\3.3.0_9757\experiment--2015_03_25_14_05_40\slide--S00\chamber--U00--V00\field--X00--Y00\image--L0000--S00--U00--V00--J133--E00--O00--X00--Y00--T0000--Z00--C01.ome.tif
Scanning well (0,1)
2015-03-25 15:07:31.042606
\3.3.0_9757\experiment--2015_03_25_14_07_34\slide--S00\chamber--U00--V00\field--X00--Y00\image--L0000--S00--U00--V00--J137--E00--O00--X00--Y00--T0000--Z00--C01.ome.tif
Scanning well (0,2)
2015-03-25 15:09:27.612274
\3.3.0_9757\experiment--2015_03_25_14_09_31\slide--S00\chamber--U00--V00\field--X00--Y00\image--L0000--S00--U00--V00--J141--E00--O00--X00--Y00--T0000--Z00--C01.ome.tif
Scanning well (1,7)
2015-03-25 15:11:23.331892
\3.3.0_9757\experiment--2015_03_25_14_11_26\slide--S00\chamber--U00--V00\field--X00--Y00\image--L0000--S00--U00--V00--J145--E00--O00--X00--Y00--T0000--Z00--C01.ome.tif
Scanning well (1,6)
2015-03-25 15:13:17.381416
\3.3.0_9757\experiment--2015_03_25_14_13_20\slide--S00\chamber--U00--V00\field--X00--Y00\imag

KeyboardInterrupt: 